<img src="./images/header.png">

***

# Exploiting Sentinel-1 SAR time series and artificial neural networks to detect grasslands in the northern Brazilian Amazon

[Part 2 - Data preparation]

Willian Vieira de Oliveira

<a id='Summary'></a>
### SUMMARY

1. [**Project Description**](./1_Project_Description.ipynb#About)
    1. [Primary objective](./1_Project_Description.ipynb#PrimaryObjetive)
    1. [Secondary objectives](./1_Project_Description.ipynb#SecondaryObjetives)
    
1. [**Study Site**](./1_Project_Description.ipynb#StudySite)

1. [**Sentinel-1 Data Description**](./1_Project_Description.ipynb#DataDescription)

1. [**Methodology Flowchart**](./1_Project_Description.ipynb#Methodology)
    1. [**MLP Architecture**](./1_Project_Description.ipynb#MLP)
    1. [**CNN Architecture**](./1_Project_Description.ipynb#CNN)
    1. [**LSTM Architecture**](./1_Project_Description.ipynb#LSTM)
    
1. [**Data preparation**](#Summary)

1. [**Data classification**](#Summary)
    1. [**Classification of CR data**](./3_Classification_CR.ipynb)
    1. [**Classification of NL data**](./3_Classification_NL.ipynb)
    1. [**Classification of RGI data**](./3_Classification_RGI.ipynb)
    1. [**Classification of VH data**](./3_Classification_VH.ipynb)
    1. [**Classification of VV data**](./3_Classification_VV.ipynb)
    
1. [**Results**](./4_Results_and_Conclusion.ipynb#Results)

1. [**Conclusion**](./4_Results_and_Conclusion.ipynb#Conclusion)
***

## Import required packages

In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal, ogr

***
## Input parameters

#### Image data cubes

In [31]:
filenames = ['VV', 'VH', 'CR', 'NL', 'RGI']
directories = ["DATA/VV_Cube.tif", "DATA/VH_Cube.tif", "DATA/CR_Cube.tif", "DATA/NL_Cube.tif", "DATA/RGI_Cube.tif"]

dir_output = "OUTPUT/"

df_columns = ['2017-09-22', '2017-10-04','2017-10-16','2017-10-28','2017-11-09','2017-11-21','2017-12-03','2017-12-15',
           '2017-12-27','2018-01-08','2018-01-20','2018-02-01','2018-02-13','2018-02-25','2018-03-09','2018-03-21',
           '2018-04-02','2018-04-14','2018-04-26','2018-05-08','2018-05-20','2018-06-01','2018-06-13','2018-06-25',
           '2018-07-07','2018-07-19','2018-07-31','2018-08-12','2018-08-24','2018-09-05','2018-09-17']

# Would you like to write the dataframes to CSV files and the classificatio products to GeoTIFF files? ['YES', 'NO']
#write_files = 'NO'
write_files = 'YES'

#### Reading the shapefile that contains point samples related to all the three classes analysed in this study

**Obs.:** It is important to use a shapefile in the same projection of the raster image used to extract the pixel values.

In [8]:
## Shapefiles with point locations of samples related to different classes
shp_samples = "DATA/amostras_500/amostras_500.shp"

try:
    samples_points = gpd.read_file(shp_samples, encoding='utf-8')
    print("The file was read!")
except Exception as e:
    print(str(e))

The file was read!


In [9]:
samples_points.head(2)

,OBJECTID,class,geometry
0,1,0,POINT (765244.8964999998 9696722.7234)
1,2,0,POINT (755975.7882000003 9678883.2358)


#### Identification of the column related to the class identifier

In [10]:
# [0: First Column, 1: Second, 2...]
class_column = 1

***
## AUXILIARY FUNCTIONS

#### Extract samples from image using point locations

In [11]:
def ExtractSamples(raster, header_dates, shp, class_column):
    #create dataframe
    df = pd.DataFrame(columns=header_dates)
    df["class"] = None
    
    df_temp = pd.DataFrame(np.nan, index=range(1), columns=header_dates)
    
    src_ds = gdal.Open(raster)
    gt = src_ds.GetGeoTransform()
    
    Nbands = src_ds.RasterCount
      
    ds = ogr.Open(shp)
    lyr = ds.GetLayer()
    
    row = 0
    for feat in lyr:
        geom = feat.GetGeometryRef()
        mx, my = geom.GetX(), geom.GetY() # coord in map units
        
        # Convert from map to pixel coordinates
        # Only works for geotransforms with no rotation
        px = int((mx - gt[0]) / gt[1]) # x pixel
        py = int((my - gt[3]) / gt[5]) # y pixel
        
        #df_temp['X'].loc[0] = float(mx)
        #df_temp['Y'].loc[0] = float(my)
        
        #df_temp["class"].loc[0] = feat[class_column]
        
        for band in range(0, Nbands):
            rb = src_ds.GetRasterBand(band+1)
            intval = rb.ReadAsArray(px, py, 1, 1)
            
            df_temp[header_dates[band]].loc[0] = float(intval[0]) #### this is the value of the pixel, forcing it to a float 
            
        
        df.loc[row] = df_temp.loc[0]
        df["class"].loc[row] = feat[class_column]
        row = row + 1
        
    # Closing files
    src_ds = None
    ds = None
    
    return df

#### Extraction of temporal metrics

In [12]:
def ComputeMetrics(df, option):
    if (option == "samples"):
        # Header of the new dataframe
        metrics_header = np.array(['Mean', 'Std', 'Sum', 'Min', 'Max', 'Amplitude', 'CoefVariation', 'Class'])
        df_classes = df['class'].copy()
        df.drop('class', axis=1, inplace=True)
    
    else: # Extracting metrics for pixels instead of samples
        metrics_header = np.array(['Mean', 'Std', 'Sum', 'Min', 'Max', 'Amplitude', 'CoefVariation'])  
    
    # Dataframe, composed only by the header
    df_metrics = pd.DataFrame(columns=metrics_header)
    
    # Metrics
    df_metrics['Mean'] = df.apply(lambda row : row.mean(), axis = 1)
    df_metrics['Std'] = df.apply(lambda row : row.std(), axis = 1)
    df_metrics['Sum'] = df.apply(lambda row : row.sum(), axis = 1)
    df_metrics['Min'] = df.apply(lambda row : row.min(), axis = 1)
    df_metrics['Max'] = df.apply(lambda row : row.max(), axis = 1)
    df_metrics['Amplitude'] = df.apply(lambda row : row.max()-row.min(), axis = 1)
    df_metrics['CoefVariation'] = df.apply(lambda row : row.std()/row.mean(), axis = 1)
    
    if (option == "samples"):
        df_metrics['Class'] = df_classes
            
    return df_metrics

#### Write a dataframe to CSV file

In [42]:
def WriteCSV(df, filename):
    try:
        df.to_csv(filename, sep=',', index=False, encoding='utf-8-sig') # using 'utf-8-sig' encoding 
                                                                        #improves efficiency to open it on Excel.
        print("    The dataframe was written to file!")
    except Exception as e:
        print(str(e))

#### Open an image

In [14]:
def openImage(filepath):
    data = gdal.Open(filepath)
    return data

***
## DATA PREPARATION FOR CLASSIFICATION


### Time series extraction for sample locations (from shapefile, point locations)

In [43]:
dataframes_samplesTS = []

for i, (filename, directory) in enumerate(zip(filenames, directories)):
    print("Extracting TS samples from the: ", filename, " data cube...")
    df_samplesTS = ExtractSamples(directory, df_columns, shp_samples, class_column)
    df_samplesTS['class'] = df_samplesTS['class'].apply(int) # convert column 'class' from float to int
    
    # Writing the dataframe to CSV file
    if (write_files == 'YES'):
        WriteCSV(df_samplesTS, dir_output+'TimeSeries_AllSamples_'+filename+'.csv')
    
    dataframes_samplesTS.append(df_samplesTS)

Extracting TS samples from the:  VV  data cube...
    The dataframe was written to file!
Extracting TS samples from the:  VH  data cube...
    The dataframe was written to file!
Extracting TS samples from the:  CR  data cube...
    The dataframe was written to file!
Extracting TS samples from the:  NL  data cube...
    The dataframe was written to file!
Extracting TS samples from the:  RGI  data cube...
    The dataframe was written to file!


##### Example: Dataframe obtained for the VH data cube:

In [44]:
df_samplesTS_VH = dataframes_samplesTS[1]
df_samplesTS_VH.head()

,2017-09-22,2017-10-04,2017-10-16,2017-10-28,2017-11-09,2017-11-21,2017-12-03,2017-12-15,2017-12-27,2018-01-08,...,2018-06-13,2018-06-25,2018-07-07,2018-07-19,2018-07-31,2018-08-12,2018-08-24,2018-09-05,2018-09-17,class
0,0.046227,0.052834,0.052655,0.044659,0.045854,0.048045,0.048873,0.052265,0.067018,0.044270,...,0.057443,0.052766,0.048808,0.055010,0.055540,0.053429,0.053200,0.042548,0.049838,0
1,0.057436,0.069680,0.061249,0.056792,0.062407,0.056256,0.052711,0.088279,0.089773,0.077882,...,0.068463,0.075454,0.076519,0.087927,0.072986,0.063441,0.059012,0.065404,0.063618,0
2,0.054750,0.044581,0.052402,0.049010,0.052049,0.048011,0.053907,0.067100,0.075462,0.071251,...,0.081516,0.063253,0.063312,0.062837,0.063550,0.049705,0.054140,0.055232,0.056488,0
3,0.036321,0.033173,0.029707,0.029013,0.032870,0.033574,0.030071,0.038529,0.049874,0.039385,...,0.045296,0.044558,0.039028,0.043760,0.044582,0.037811,0.035878,0.031677,0.036693,0
4,0.034539,0.036610,0.037197,0.043403,0.039930,0.034697,0.034381,0.049837,0.055737,0.046845,...,0.053660,0.052796,0.046458,0.046275,0.053157,0.047727,0.042873,0.054173,0.046401,0


### Extraction of temporal metrics regarding the time series of each sample location

In [46]:
dataframes_metrics_samples = []

for i, (filename, df_samplesTS) in enumerate(zip(filenames, dataframes_samplesTS)):
    print("Extracting temporal metrics from the: ", filename, " dataframe...")
    df_metrics_samples = ComputeMetrics(df_samplesTS.copy(), 'samples')
    
    # Writing the dataframe to CSV file
    if (write_files == 'YES'):
        WriteCSV(df_metrics_samples, dir_output+'Metrics_AllSamples_'+filename+'.csv')
    
    dataframes_metrics_samples.append(df_metrics_samples)

Extracting temporal metrics from the:  VV  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  VH  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  CR  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  NL  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  RGI  dataframe...
    The dataframe was written to file!


##### Example: Metrics extracted from the VH data:

In [47]:
df_metrics_samples = dataframes_metrics_samples[1]
df_metrics_samples.head()

,Mean,Std,Sum,Min,Max,Amplitude,CoefVariation,Class
0,0.052776,0.005973,1.636049,0.041501,0.067018,0.025516,0.113180,0
1,0.072890,0.010835,2.259599,0.052711,0.090519,0.037808,0.148650,0
2,0.061596,0.009167,1.909474,0.044581,0.081516,0.036935,0.148825,0
3,0.039385,0.006330,1.220950,0.029013,0.051529,0.022516,0.160728,0
4,0.047505,0.007001,1.472654,0.034381,0.059871,0.025491,0.147364,0


***
### Time series extraction for all pixels

In [48]:
dataframes_TimeSeries_AllPixels = []

for i, (filename, directory) in enumerate(zip(filenames, directories)):
    print("Extracting the TS for all pixels of the: ", filename, " data cube...")

    datacube = openImage(directory)

    Nrows = datacube.RasterYSize - 6 # We do not consider border pixels. We removed both the first and the last three rows.
    Ncols = datacube.RasterXSize - 6 # We do not consider border pixels. We removed both the first and the last three columns.
    Nbands = datacube.RasterCount

    arr = datacube.ReadAsArray(3, 3, Ncols, Nrows) # xoff, yoff, xcount, ycount
    datacube = None
    
    df_list = []
    for band in range(0, Nbands):
        array = arr[band].flatten()
        df = pd.DataFrame(array, columns=[df_columns[band]])
        df_list.append(df)

    df_datacube = pd.concat(df_list, axis=1)

    # Writing the dataframe to CSV file
    if (write_files == 'YES'):
        WriteCSV(df_datacube, dir_output+'TimeSeries_AllPixels_'+filename+'.csv')
        
    dataframes_TimeSeries_AllPixels.append(df_datacube)

Extracting the TS for all pixels of the:  VV  data cube...
    The dataframe was written to file!
Extracting the TS for all pixels of the:  VH  data cube...
    The dataframe was written to file!
Extracting the TS for all pixels of the:  CR  data cube...
    The dataframe was written to file!
Extracting the TS for all pixels of the:  NL  data cube...
    The dataframe was written to file!
Extracting the TS for all pixels of the:  RGI  data cube...
    The dataframe was written to file!


##### Example: Dataframe obtained for the VH data cube:

In [49]:
df_datacube_VH = dataframes_TimeSeries_AllPixels[1]
df_datacube_VH.head()

,2017-09-22,2017-10-04,2017-10-16,2017-10-28,2017-11-09,2017-11-21,2017-12-03,2017-12-15,2017-12-27,2018-01-08,...,2018-06-01,2018-06-13,2018-06-25,2018-07-07,2018-07-19,2018-07-31,2018-08-12,2018-08-24,2018-09-05,2018-09-17
0,0.030140,0.035049,0.026826,0.037771,0.032626,0.028671,0.028586,0.035451,0.043515,0.044464,...,0.038554,0.033150,0.029756,0.033361,0.039310,0.039912,0.031309,0.038160,0.030636,0.035158
1,0.030385,0.033345,0.027814,0.036575,0.032115,0.029239,0.028343,0.037187,0.043729,0.042603,...,0.038298,0.032843,0.029729,0.034149,0.038422,0.037959,0.030161,0.038267,0.030045,0.034407
2,0.026909,0.029020,0.024550,0.030759,0.028446,0.025149,0.025230,0.032081,0.038999,0.036744,...,0.034003,0.029203,0.028400,0.031042,0.033451,0.032695,0.025740,0.033955,0.026470,0.029883
3,0.037398,0.040440,0.034533,0.040932,0.038193,0.034530,0.032443,0.046084,0.052855,0.051553,...,0.047204,0.041964,0.041130,0.048807,0.047363,0.044682,0.036132,0.049332,0.038773,0.042218
4,0.041544,0.043606,0.039519,0.042680,0.042706,0.037016,0.033596,0.049529,0.057144,0.053213,...,0.052151,0.047348,0.044832,0.052455,0.051900,0.048358,0.040474,0.050013,0.039172,0.046000


### Extraction of temporal metrics for all pixels

In [50]:
dataframes_metrics_pixels = []

for i, (filename, df_datacube) in enumerate(zip(filenames, dataframes_TimeSeries_AllPixels)):
    print("Extracting temporal metrics from the: ", filename, " dataframe...")

    df_metrics_pixels = ComputeMetrics(df_datacube.copy(), 'pixels')
    
    # Writing the dataframe to CSV file
    if (write_files == 'YES'):
        WriteCSV(df_metrics_pixels, dir_output+'Metrics_AllPixels_'+filename+'.csv')
    
    dataframes_metrics_pixels.append(df_metrics_pixels)

Extracting temporal metrics from the:  VV  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  VH  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  CR  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  NL  dataframe...
    The dataframe was written to file!
Extracting temporal metrics from the:  RGI  dataframe...
    The dataframe was written to file!


##### Example: Metrics extracted from the VH data:

In [51]:
df_metrics_pixels = dataframes_metrics_pixels[1]
df_metrics_pixels.head()

,Mean,Std,Sum,Min,Max,Amplitude,CoefVariation
0,0.035449,0.004395,1.098929,0.026826,0.044464,0.017639,0.123975
1,0.035027,0.004136,1.085838,0.027814,0.043729,0.015914,0.118093
2,0.030636,0.003566,0.949714,0.024550,0.038999,0.014448,0.116413
3,0.042832,0.005302,1.327800,0.032443,0.052855,0.020412,0.123796
4,0.045681,0.005602,1.416098,0.033596,0.057144,0.023548,0.122624
